In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.float_format', lambda x: '%.1f' % x if not pd.isnull(x) else ' ')  # Display 1 decimal place, and empty for NaN
pd.set_option('expand_frame_repr', False)  # This line helps to display all columns

# define variables for past 5 years
ebit = np.array([60, 66, 72, 78, 85]) 
tax_rate = np.array([0.3, 0.3, 0.3, 0.3, 0.3]) 
capex = np.array([30, 33, 36, 40, 44])
depreciation = np.array([10, 11, 12, 13, 14])
roic = np.array([0.2, 0.21, 0.22, 0.23, 0.24])
risk_free_rate = 0.02 
equity_risk_premium = 0.05 
wacc = 0.08
revenue = np.array([100, 110, 120, 130, 140]) 


In [2]:
# calculate reinvestment rate and free cash flows for past 5 years
reinvestment_rate = (capex - depreciation) / (ebit * (1 - tax_rate))
ebit_margin = ebit / revenue

# calculate growth rate for the first 5 years
growth_rate = reinvestment_rate * roic

In [3]:
# create DataFrame to store all the data
columns = ['Year ' + str(i) for i in range(-4, 11)] + ['Terminal Value']
index = ['Revenue', 'Revenue Growth Rate (%)', 'EBIT', 'EBIT Margin (%)', 'EBIT (1 - tax rate)', 'Reinvestment', 'Free Cash Flows to Firm', 'Discount Factor', 'PV of Free Cash Flows']
df = pd.DataFrame(columns=columns, index=index)

In [4]:
# create forecast for the next 10 years
years = np.arange(1, 11)
growth_rates = np.linspace(growth_rate[-1], risk_free_rate, 10)
wacc_values = np.linspace(wacc, risk_free_rate + equity_risk_premium, 10)

# Historical data
for i in range(5):
    df.loc['Revenue', 'Year ' + str(i-4)] = revenue[i]
    df.loc['Revenue Growth Rate (%)', 'Year ' + str(i-4)] = (revenue[i] - revenue[i-1]) / revenue[i-1] * 100 if i != 0 else ""
    df.loc['EBIT', 'Year ' + str(i-4)] = ebit[i]
    df.loc['EBIT Margin (%)', 'Year ' + str(i-4)] = ebit_margin[i] * 100
    df.loc['EBIT (1 - tax rate)', 'Year ' + str(i-4)] = ebit[i] * (1 - tax_rate[i])
    df.loc['Reinvestment', 'Year ' + str(i-4)] = reinvestment_rate[i] * ebit[i]
    df.loc['Free Cash Flows to Firm', 'Year ' + str(i-4)] = ebit[i] * (1 - tax_rate[i]) - reinvestment_rate[i] * ebit[i]

# calculate revenue, EBIT, and other values for the next 10 years
for i in range(10):
    if i == 0:
        df.loc['Revenue', 'Year ' + str(i+1)] = revenue[-1] * (1 + growth_rates[i])
        df.loc['Revenue Growth Rate (%)', 'Year ' + str(i+1)] = growth_rates[i] * 100
    else:
        df.loc['Revenue', 'Year ' + str(i+1)] = df.loc['Revenue', 'Year ' + str(i)] * (1 + growth_rates[i])
        df.loc['Revenue Growth Rate (%)', 'Year ' + str(i+1)] = growth_rates[i] * 100

    df.loc['EBIT', 'Year ' + str(i+1)] = df.loc['Revenue', 'Year ' + str(i+1)] * ebit_margin[-1]
    df.loc['EBIT Margin (%)', 'Year ' + str(i+1)] = ebit_margin[-1] * 100
    df.loc['EBIT (1 - tax rate)', 'Year ' + str(i+1)] = df.loc['EBIT', 'Year ' + str(i+1)] * (1 - tax_rate[-1])
    df.loc['Reinvestment', 'Year ' + str(i+1)] = reinvestment_rate[-1] * df.loc['EBIT (1 - tax rate)', 'Year ' + str(i+1)]
    df.loc['Free Cash Flows to Firm', 'Year ' + str(i+1)] = df.loc['EBIT (1 - tax rate)', 'Year ' + str(i+1)] - df.loc['Reinvestment', 'Year ' + str(i+1)]
    df.loc['Discount Factor', 'Year ' + str(i+1)] = 1 / (1 + wacc_values[i]) ** (i+1)
    df.loc['PV of Free Cash Flows', 'Year ' + str(i+1)] = df.loc['Free Cash Flows to Firm', 'Year ' + str(i+1)] * df.loc['Discount Factor', 'Year ' + str(i+1)]

# calculate terminal values
terminal_growth_rate = risk_free_rate
terminal_reinvestment_rate = terminal_growth_rate / (risk_free_rate + equity_risk_premium)
terminal_free_cash_flows = df.loc['EBIT (1 - tax rate)', 'Year 10'] * (1 - terminal_reinvestment_rate)
terminal_value = terminal_free_cash_flows / (wacc_values[-1] - terminal_growth_rate)
df.loc['PV of Free Cash Flows', 'Terminal Value'] = terminal_value * df.loc['Discount Factor', 'Year 10']

In [5]:
# Total enterprise value is sum of all PV of cash flows
total_enterprise_value = df.loc['PV of Free Cash Flows'].sum()
df.loc['Enterprise Value'] = np.nan
df.loc['Enterprise Value', 'Year 0'] = total_enterprise_value
print('Total Enterprise Value: ', total_enterprise_value)

Total Enterprise Value:  1166.562910975288


In [6]:
df = df.replace(np.nan, '', regex=True)

In [7]:
# Convert the DataFrame to an HTML table and use CSS to add borders and padding
df_html = df.to_html()
styled_df_html = f"""
<style>
.dataframe {{border: 1px solid black; border-collapse: collapse;}}
.dataframe th, .dataframe td {{text-align: right; padding: 10px; border: 1px solid black;}}
.dataframe th:nth-child(1), .dataframe td:nth-child(1) {{text-align: left;}}
.dataframe td:nth-child(n+2):nth-child(-n+6) {{color: lightgrey;}}
.dataframe td:nth-child(23), .dataframe td:last-child {{color: black;}}
.dataframe td:last-child {{color: black;}}
</style>
{df_html}
"""

# Display the HTML table
display(HTML(styled_df_html))

,Year -4,Year -3,Year -2,Year -1,Year 0,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Year 7,Year 8,Year 9,Year 10,Terminal Value
Revenue,100,110,120,130,140,156.9,174.2,191.3,208.0,223.9,238.4,251.2,261.9,270.0,275.4,
Revenue Growth Rate (%),,10.0,9.1,8.3,7.7,12.1,11.0,9.9,8.7,7.6,6.5,5.4,4.2,3.1,2.0,
EBIT,60,66,72,78,85,95.3,105.7,116.2,126.3,135.9,144.8,152.5,159.0,164.0,167.2,
EBIT Margin (%),60.0,60.0,60.0,60.0,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,60.7,
EBIT (1 - tax rate),42.0,46.2,50.4,54.6,59.5,66.7,74.0,81.3,88.4,95.2,101.3,106.8,111.3,114.8,117.1,
Reinvestment,28.6,31.4,34.3,38.6,42.9,33.6,37.3,41.0,44.6,48.0,51.1,53.8,56.1,57.9,59.0,
Free Cash Flows to Firm,13.4,14.8,16.1,16.0,16.6,33.1,36.7,40.3,43.8,47.2,50.2,52.9,55.2,56.9,58.0,
Discount Factor,,,,,,0.9,0.9,0.8,0.7,0.7,0.6,0.6,0.6,0.5,0.5,
PV of Free Cash Flows,,,,,,30.6,31.5,32.2,32.6,32.8,32.7,32.3,31.6,30.7,29.5,850.1
Enterprise Value,,,,,1166.6,,,,,,,,,,,
